# Segundo Trabalho Prático de Introdução à Computação Visual (ICV) na UFMG:

Artur Gaspar da Silva

### Baixando video, modelo, alvo e objloader

In [ ]:
!wget https://homepages.dcc.ufmg.br/~artur.gaspar/ICV/TP2/entrada.mp4
!wget https://homepages.dcc.ufmg.br/~artur.gaspar/ICV/TP2/pikach_obj.zip
!wget https://homepages.dcc.ufmg.br/~artur.gaspar/ICV/TP2/alvo.jpg
!wget https://homepages.dcc.ufmg.br/~artur.gaspar/ICV/TP2/objloader.py

### Importando bibliotecas a serem usadas:

In [1]:
import pygame
import cv2
from cv2 import threshold
import numpy as np

from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *

from PIL import Image
from PIL import ImageOps

from objloader import *

pygame 2.1.2 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Parte 1: Calibrando a câmera

Antes de tudo, precisamos realizar o processo de calibrar a câmera, ou seja, obter seus parâmetros intrínsecos (foco, centro óptico, distorções e etc.)

Para fazer isso, utilizamos o toolbox de Jean-Yves Bouguet para Octave. Os frames utilizados para calibração foram os seguintes:

<img src="https://homepages.dcc.ufmg.br/~artur.gaspar/ICV/TP2/calibs/0.jpg" alt="Drawing" style="width: 200px;"/>
<img src="https://homepages.dcc.ufmg.br/~artur.gaspar/ICV/TP2/calibs/1.jpg" alt="Drawing" style="width: 200px;"/>
<img src="https://homepages.dcc.ufmg.br/~artur.gaspar/ICV/TP2/calibs/2.jpg" alt="Drawing" style="width: 200px;"/>
<img src="https://homepages.dcc.ufmg.br/~artur.gaspar/ICV/TP2/calibs/3.jpg" alt="Drawing" style="width: 200px;"/>
<img src="https://homepages.dcc.ufmg.br/~artur.gaspar/ICV/TP2/calibs/4.jpg" alt="Drawing" style="width: 200px;"/>
<img src="https://homepages.dcc.ufmg.br/~artur.gaspar/ICV/TP2/calibs/5.jpg" alt="Drawing" style="width: 200px;"/>
<img src="https://homepages.dcc.ufmg.br/~artur.gaspar/ICV/TP2/calibs/6.jpg" alt="Drawing" style="width: 200px;"/>

Depois de selecionar as quinas dos tabuleiros de xadrez e utilizar o Toolkit, obtemos os seguintes resultados:
```
Calibration results after optimization (with uncertainties):

Focal Length:          fc = [ 397.32165   389.18911 ] +/- [ 31.21264   27.35409 ]
Principal point:       cc = [ 327.02195   248.75539 ] +/- [ 8.57489   17.54686 ]
Skew:             alpha_c = [ 0.00000 ] +/- [ 0.00000  ]   => angle of pixel axes = 90.00000 +/- 0.00000 degrees
Distortion:            kc = [ -0.03318   0.09094   0.02252   0.00253  0.00000 ] +/- [ 0.09034   0.18810   0.01571   0.01322  0.00000 ]
Pixel error:          err = [ 1.66291   1.57933 ]

Note: The numerical errors are approximately three times the standard deviations (for reference).


Recommendation: Some distortion coefficients are found equal to zero (within their uncertainties).
                To reject them from the optimization set est_dist=[0;0;1;1;0] and run Calibration
```

Assim nossa matriz de parâmetros intrínsecos e nossos coeficientes de distorção serão:

In [ ]:
cameraMatrix = np.array(
    [
        [397.32165, 0        , 327.02195],
        [0        , 389.18911, 248.75539],
        [0        , 0        , 1        ],
    ], dtype = np.float64
)

distCoeffs = np.array([-0.03318, 0.09094, 0.02252, 0.00253, 0.00000], dtype = np.float64)

## Parte 2: Detecção de Alvos:

Agora, precisamos detectar os alvos e suas orientações nos frames do vídeo. Para isso, primeiro vamos ler o primeiro frame do vídeo:

In [4]:
vidcap = cv2.VideoCapture('entrada.mp4')
success,image = vidcap.read()
assert(success)
assert(image.max() <= 255 and image.min() >= 0)
surf = pygame.surfarray.make_surface(image)
imagepygame = pygame.image.tostring(surf, 'RGBA', 1)
ix, iy = surf.get_rect().size

cv2.imshow('a',image)
cv2.waitKey()
cv2.destroyAllWindows()